In [1]:
import pandas as pd
import numpy as np
import scipy
import tensorflow as tf

## Load the data and explore 

In [2]:
# importing the data and prinitng few rows to examine the data
# read the CSV file 
df = pd.read_csv("codeset.csv", header=None) 
   
# print the first five rows 
print(df.head()) 

      0     1           2                                     3    4    5   \
0  37013  AFIB   314208002             Rapid atrial fibrillation  NaN  NaN   
1  37014  Afib  3320796013                        Atrial flutter  NaN  NaN   
2  37015  Afib  3322864015               Atypical atrial flutter  NaN  NaN   
3  37016  AFib   421232012   AF - Paroxysmal atrial fibrillation  NaN  NaN   
4  37017  Afib   421233019  PAF - Paroxysmal atrial fibrillation  NaN  NaN   

    6   7   8   9  10         11  12  13  14    15 16  \
0  NaN NaN NaN NaN  f  SNOMED-CT  74 NaN NaN  4010  f   
1  NaN NaN NaN NaN  f  SNOMED-CT  74 NaN NaN  4010  f   
2  NaN NaN NaN NaN  f  SNOMED-CT  74 NaN NaN  4010  f   
3  NaN NaN NaN NaN  f  SNOMED-CT  74 NaN NaN  4010  f   
4  NaN NaN NaN NaN  f  SNOMED-CT  74 NaN NaN  4010  f   

                                     17  
0  3137fbd7-551e-4e7e-96f8-13b12a867098  
1  3137fbd7-551e-4e7e-96f8-13b12a867098  
2  3137fbd7-551e-4e7e-96f8-13b12a867098  
3  3137fbd7-551e-4e7

In [3]:
print(df.shape)

(32413, 18)


In [4]:
# selecting only the columns present in the problem statement
select_columns_index = [3,15]
df = df.iloc[:,select_columns_index]

In [5]:
print(df.shape)

(32413, 2)


In [6]:
print(df.head()) 

                                     3     15
0             Rapid atrial fibrillation  4010
1                        Atrial flutter  4010
2               Atypical atrial flutter  4010
3   AF - Paroxysmal atrial fibrillation  4010
4  PAF - Paroxysmal atrial fibrillation  4010


# Preprocessing and cleaning the data

### Check for any empty cells 
Exmaine the data for any empty entries and remove them 

In [7]:
# prinitng to see if any erros in data
df.iloc[600:625,:]

,3,15
600,Type 1 diabetes mellitus with mild nonprolifer...,NaN
601,Type 1 diabetes mellitus with mild nonprolifer...,NaN
602,Type 1 diabetes mellitus with moderate nonprol...,NaN
603,Type 1 diabetes mellitus with moderate nonprol...,NaN
604,Type 1 diabetes mellitus with moderate nonprol...,NaN
605,Type 1 diabetes mellitus with moderate nonprol...,NaN
606,Type 1 diabetes mellitus with moderate nonprol...,NaN
607,Type 1 diabetes mellitus with moderate nonprol...,NaN
608,Type 1 diabetes mellitus with moderate nonprol...,NaN
609,Type 1 diabetes mellitus with moderate nonprol...,NaN


In [8]:
# dropping the rows with empty values i.e. incomplete data
df = df.dropna()

In [9]:
print(df.shape)

(32052, 2)


### Check for any class frequency 
In the output of the below cell it can be observed that there are imbalances in the classes 

In [10]:
# checking number of classes present
print(df[15].value_counts(dropna=False))

M3      5382
M13     4045
M156    2305
M10     2096
6522    1732
        ... 
7542       1
7300       1
7105       1
7380       1
7030       1
Name: 15, Length: 200, dtype: int64


In [58]:
print(df["code"].value_counts(dropna=False)[:50])

M3      5382
M13     4045
M156    2305
M10     2096
6522    1732
M14     1136
M80      782
M68      750
M77      729
4000     681
M107     603
M15      592
M146     484
M155     453
4040     452
4264     349
M59      336
M91      332
M86      331
E056     331
M11      326
M157     305
M106     300
4170     300
M152     269
4160     266
M70      258
M92      240
4220     237
M67      211
6700     205
M83      202
E032     192
M145     189
E015     184
E001     170
M108     170
E007     151
M147     139
4030     121
M33      103
4210     103
M81      102
4060     102
E016     100
4090      98
4240      87
4150      86
M90       79
M115      77
Name: code, dtype: int64


In [11]:
# # reducing the problem to 10 classes
# value_counts = df[15].value_counts() # Specific column 
# threshold = 650
# to_remove = value_counts[value_counts <= threshold].index
# for code in to_remove:
#     df = df[df[15] != code]

In [12]:
print(df[15].value_counts())

M3      5382
M13     4045
M156    2305
M10     2096
6522    1732
        ... 
7542       1
7300       1
7105       1
7380       1
7030       1
Name: 15, Length: 200, dtype: int64


In [13]:
print(df.shape)

(32052, 2)


In [14]:
# renaming the cloumns for better readability 
df = df.rename(columns={3: "symptoms", 15: "code"})
df.head()

,symptoms,code
0,Rapid atrial fibrillation,4010
1,Atrial flutter,4010
2,Atypical atrial flutter,4010
3,AF - Paroxysmal atrial fibrillation,4010
4,PAF - Paroxysmal atrial fibrillation,4010


In [15]:
df["label"] = np.nan
print(df.shape)

value_counts = df["code"].value_counts() # Specific column 
code_index   = value_counts.index
print(code_index)
for i, code_id in enumerate(code_index):
    df.loc[df[df["code"] == code_id].index,"label"] = i

(32052, 3)
Index(['M3', 'M13', 'M156', 'M10', '6522', 'M14', 'M80', 'M68', 'M77', '4000',
       ...
       'M158', '7220', '7040', '7310', '7370', '7542', '7300', '7105', '7380',
       '7030'],
      dtype='object', length=200)


In [16]:
print(df.shape)
df.head()

(32052, 3)


,symptoms,code,label
0,Rapid atrial fibrillation,4010,60.0
1,Atrial flutter,4010,60.0
2,Atypical atrial flutter,4010,60.0
3,AF - Paroxysmal atrial fibrillation,4010,60.0
4,PAF - Paroxysmal atrial fibrillation,4010,60.0


In [17]:
#checking if the labels are correct
print(df["label"].value_counts())

0.0      5382
1.0      4045
2.0      2305
3.0      2096
4.0      1732
         ... 
179.0       1
193.0       1
178.0       1
186.0       1
199.0       1
Name: label, Length: 200, dtype: int64


In [18]:
# train and test split https://stackoverflow.com/questions/24147278/how-do-i-create-test-and-train-samples-from-one-dataframe-with-pandas

def split_to_train_test(df, label_column, train_frac=0.8):
    train_df, test_df = pd.DataFrame(), pd.DataFrame()
    labels = df[label_column].unique()
    for lbl in labels:
        lbl_df = df[df[label_column] == lbl]
        lbl_train_df = lbl_df.sample(frac=train_frac)
        lbl_test_df = lbl_df.drop(lbl_train_df.index)
        #print('\n%s:\n---------\ntotal:%d\ntrain_df:%d\ntest_df:%d' % (lbl, len(lbl_df), len(lbl_train_df), len(lbl_test_df)))
        train_df = train_df.append(lbl_train_df)
        test_df = test_df.append(lbl_test_df)

    return train_df, test_df


In [19]:
train_df, test_df = split_to_train_test(df, "label", train_frac=0.8)

In [20]:
#print(train_df["label"].value_counts(), test_df["label"].value_counts())
print(train_df.shape, test_df.shape)

(25651, 3) (6401, 3)


In [21]:
from sklearn.feature_extraction.text import CountVectorizer

In [22]:
vectorizer = CountVectorizer(min_df=0)
vectorizer.fit(train_df["symptoms"])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=0,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [23]:
vectorizer.vocabulary_

{'history': 1319,
 'of': 1949,
 'maze': 1689,
 'procedure': 2230,
 'for': 1150,
 'atrial': 362,
 'fibrillation': 1116,
 'persistent': 2087,
 'chronic': 618,
 'af': 190,
 'typical': 2782,
 'flutter': 1140,
 'unspecified': 2802,
 'with': 2901,
 'rapid': 2294,
 'ventricular': 2852,
 'response': 2348,
 'atypical': 370,
 'paroxysmal': 2034,
 'controlled': 713,
 'preexcited': 2191,
 'disorder': 893,
 'lone': 1624,
 'and': 249,
 'ecg': 954,
 'permanent': 2085,
 'non': 1891,
 'rheumatic': 2372,
 'transient': 2735,
 'cerebral': 576,
 'ischemia': 1511,
 'due': 923,
 'to': 2704,
 'percutaneous': 2068,
 'transluminal': 2736,
 'ablation': 131,
 'wall': 2887,
 'paf': 2006,
 '427': 84,
 '31': 60,
 'polyhydramnios': 2150,
 'first': 1124,
 'trimester': 2761,
 'not': 1914,
 'applicable': 299,
 'or': 1972,
 'infections': 1440,
 'urethra': 2811,
 'in': 1416,
 'pregnancy': 2197,
 'maternal': 1681,
 'care': 551,
 'suspected': 2595,
 'hereditary': 1308,
 'disease': 890,
 'fetus': 1112,
 'other': 1990,
 'isoi

In [24]:
print(len(vectorizer.get_feature_names()))

2931


In [25]:
# convert train and test data
train_data = vectorizer.transform(train_df["symptoms"]).toarray()
test_data  = vectorizer.transform(test_df["symptoms"]).toarray()
print(train_data.shape, test_data.shape)

(25651, 2931) (6401, 2931)


In [26]:
#
train_labels = train_df["label"].to_numpy()
test_labels = test_df["label"].to_numpy()
print(train_labels.shape, test_labels.shape)

(25651,) (6401,)


In [27]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer()

In [28]:
train_data_tfidf = transformer.fit_transform(train_data)
test_data_tfidf  = transformer.fit_transform(test_data)
print(train_data_tfidf.shape, test_data_tfidf.shape)

(25651, 2931) (6401, 2931)


In [29]:
train_data_tfidf = scipy.sparse.csr_matrix.toarray(train_data_tfidf)
test_data_tfidf  = scipy.sparse.csr_matrix.toarray(test_data_tfidf)
print(train_data_tfidf.shape, test_data_tfidf.shape)

(25651, 2931) (6401, 2931)


In [30]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(train_data_tfidf, train_labels)
score = classifier.score(test_data_tfidf, test_labels)

print("Accuracy:", score)

Accuracy: 0.8200281206061553


/home/chaitanya/VirtualEnv/env_py3/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [71]:
model = tf.keras.models.Sequential([
  
  tf.keras.layers.Dense(200, activation='softmax')
])

# tf.keras.layers.Dense(512, activation='relu'),
#   tf.keras.layers.Dense(512, activation='relu'),
#   tf.keras.layers.Dense(256, activation='relu'),


In [72]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [73]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [75]:
model.fit(train_data_tfidf, train_labels, epochs=50)

Train on 25651 samples
Epoch 1/50
25651/25651 [==============================] - 3s 109us/sample - loss: 4.4967 - accuracy: 0.8213
Epoch 2/50
25651/25651 [==============================] - 3s 107us/sample - loss: 4.4961 - accuracy: 0.8216
Epoch 3/50
25651/25651 [==============================] - 3s 108us/sample - loss: 4.4954 - accuracy: 0.8222
Epoch 4/50
25651/25651 [==============================] - 3s 108us/sample - loss: 4.4948 - accuracy: 0.8225
Epoch 5/50
25651/25651 [==============================] - 3s 108us/sample - loss: 4.4942 - accuracy: 0.8227
Epoch 6/50
25651/25651 [==============================] - 3s 109us/sample - loss: 4.4936 - accuracy: 0.8231
Epoch 7/50
25651/25651 [==============================] - 3s 110us/sample - loss: 4.4930 - accuracy: 0.8237
Epoch 8/50
25651/25651 [==============================] - 3s 108us/sample - loss: 4.4925 - accuracy: 0.8239
Epoch 9/50
25651/25651 [==============================] - 3s 109us/sample - loss: 4.4919 - accuracy: 0.8243
Epoch

In [45]:
model.evaluate(test_data_tfidf,  test_labels, verbose=2)

6401/1 - 0s - loss: 4.9734 - accuracy: 0.6666


[4.640062368506474, 0.6666146]